# Dependencies

In [16]:
import sys
sys.path.append("../")
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from src.utils import build_freqs

# Preprocessing

In [8]:
df = pd.read_csv("../data/raw/masakhane_afrisenti_twi_train.csv")
df.head()

,tweet,label
0,kako be shark but wo ti ewu,negative
1,br ne bayie nti na me supporti man city,negative
2,s3 woofis3 mada wafutuo tantan no 3y3wo s3mafa...,negative
3,wabɔdam anaa wo trumu yɛ nkate nkwan aseɛ,negative
4,enfa bi da bra 🤣🤣,negative


In [5]:
encoder = LabelEncoder()

In [10]:
df['sentiment'] = encoder.fit_transform(df['label'])
print(encoder.classes_)

['negative' 'neutral' 'positive']


In [12]:
df.head()

,tweet,label,sentiment
0,kako be shark but wo ti ewu,negative,0
1,br ne bayie nti na me supporti man city,negative,0
2,s3 woofis3 mada wafutuo tantan no 3y3wo s3mafa...,negative,0
3,wabɔdam anaa wo trumu yɛ nkate nkwan aseɛ,negative,0
4,enfa bi da bra 🤣🤣,negative,0


In [13]:
train_x = df['tweet']
train_y = df['sentiment']

# Model Training

In [17]:
freqs = build_freqs(train_x,train_y)